In [ ]:
# | default_exp _application.tester

In [ ]:
# | export

import asyncio
import inspect
from contextlib import asynccontextmanager
from typing import *

from pydantic import BaseModel

from fastkafka._application.app import FastKafka
from fastkafka._components.meta import delegates, export, patch
from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker
from fastkafka._testing.in_memory_broker import InMemoryBroker
from fastkafka._testing.local_redpanda_broker import LocalRedpandaBroker

In [ ]:
import pytest
from pydantic import Field

from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

## Fastkafka Tester class

In [ ]:
# | export


@export("fastkafka.testing")
class Tester(FastKafka):
    __test__ = False

    @delegates(ApacheKafkaBroker.__init__)
    def __init__(
        self,
        app: Union[FastKafka, List[FastKafka]],
        *,
        broker: Optional[
            Union[ApacheKafkaBroker, LocalRedpandaBroker, InMemoryBroker]
        ] = None,
    ):
        """Mirror-like object for testing a FastFafka application

        Can be used as context manager

        """
        self.apps = app if isinstance(app, list) else [app]
        host, port = self.apps[0]._kafka_config["bootstrap_servers"].split(":")
        super().__init__(kafka_brokers={"localhost": {"url": host, "port": port}})
        self.create_mirrors()

        self.broker = broker

    @delegates(LocalRedpandaBroker.__init__)
    def using_local_redpanda(self, **kwargs: Any) -> "Tester":
        """Starts local Redpanda broker used by the Tester instance

        Args:
            listener_port: Port on which the clients (producers and consumers) can connect
            tag: Tag of Redpanda image to use to start container
            seastar_core: Core(s) to use byt Seastar (the framework Redpanda uses under the hood)
            memory: The amount of memory to make available to Redpanda
            mode: Mode to use to load configuration properties in container
            default_log_level: Log levels to use for Redpanda
            topics: List of topics to create after sucessfull redpanda broker startup
            retries: Number of retries to create redpanda service
            apply_nest_asyncio: set to True if running in notebook
            port allocation if the requested port was taken

        Returns:
            An instance of tester with Redpanda as broker
        """
        topics = set().union(*(app.get_topics() for app in self.apps))
        kwargs["topics"] = (
            topics.union(kwargs["topics"]) if "topics" in kwargs else topics
        )
        self.broker = LocalRedpandaBroker(**kwargs)

        return self

    @delegates(ApacheKafkaBroker.__init__)
    def using_local_kafka(self, **kwargs: Any) -> "Tester":
        """Starts local Kafka broker used by the Tester instance

        Args:
            data_dir: Path to the directory where the zookeepeer instance will save data
            zookeeper_port: Port for clients (Kafka brokes) to connect
            listener_port: Port on which the clients (producers and consumers) can connect
            topics: List of topics to create after sucessfull Kafka broker startup
            retries: Number of retries to create kafka and zookeeper services using random
            apply_nest_asyncio: set to True if running in notebook
            port allocation if the requested port was taken

        Returns:
            An instance of tester with Kafka as broker
        """
        topics = set().union(*(app.get_topics() for app in self.apps))
        kwargs["topics"] = (
            topics.union(kwargs["topics"]) if "topics" in kwargs else topics
        )
        self.broker = ApacheKafkaBroker(**kwargs)

        return self

    async def _start_tester(self) -> None:
        """Starts the Tester"""
        for app in self.apps:
            app.create_mocks()
            await app.__aenter__()
        self.create_mocks()
        await super().__aenter__()
        await asyncio.sleep(3)

    async def _stop_tester(self) -> None:
        """Shuts down the Tester"""
        await super().__aexit__(None, None, None)
        for app in self.apps[::-1]:
            await app.__aexit__(None, None, None)

    def create_mirrors(self) -> None:
        pass

    @asynccontextmanager
    async def _create_ctx(self) -> AsyncGenerator["Tester", None]:
        if self.broker is None:
            topics = set().union(*(app.get_topics() for app in self.apps))
            self.broker = InMemoryBroker()

        bootstrap_server = await self.broker._start()
        old_bootstrap_servers: List[str] = list()
        try:
            if isinstance(self.broker, (ApacheKafkaBroker, LocalRedpandaBroker)):
                self._set_bootstrap_servers(bootstrap_servers=bootstrap_server)
                for app in self.apps:
                    old_bootstrap_servers.append(app._kafka_config["bootstrap_servers"])
                    app._set_bootstrap_servers(bootstrap_server)
            await self._start_tester()
            try:
                yield self
            finally:
                await self._stop_tester()
        finally:
            await self.broker._stop()
            for app, server in zip(self.apps, old_bootstrap_servers):
                app._set_bootstrap_servers(server)

    async def __aenter__(self) -> "Tester":
        self._ctx = self._create_ctx()
        return await self._ctx.__aenter__()

    async def __aexit__(self, *args: Any) -> None:
        await self._ctx.__aexit__(*args)

In [ ]:
for _ in range(2):
    with pytest.raises(RuntimeError) as e:
        async with Tester(app) as tester:
            assert tester.broker.is_started
            assert tester.is_started
            raise RuntimeError("ok")

    print(e)
    assert not tester.broker.is_started
    assert not tester.is_started

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'localhost:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] fastk

In [ ]:
tester_with_redpanda = Tester(app).using_local_redpanda(tag="v22.3.15")
assert isinstance(tester_with_redpanda.broker, LocalRedpandaBroker)

In [ ]:
tester = Tester(app)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(1)
    tester.mocks.on_predictions.assert_called()

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'localhost:9092', 'auto_offset_reset': 'earliest', '

AssertionError: Expected 'mock' to have been called.

In [ ]:
tester = Tester(app).using_local_kafka(zookeeper_port=9998, listener_port=9788)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

In [ ]:
# | export


def mirror_producer(topic: str, producer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(producer_f).return_annotation

    async def skeleton_func(msg: BaseModel) -> None:
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "on_" + topic

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig  # type: ignore

    return mirror_func

In [ ]:
@app.produces()
def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
def some_log(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, _) in app._producers_store.items():
    mirror = mirror_producer(topic, producer_f)
    assert mirror.__name__ == "on_" + topic
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export


def mirror_consumer(topic: str, consumer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    async def skeleton_func(msg: BaseModel) -> BaseModel:
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "to_" + topic

    # adjust arg and return val
    sig = sig.replace(parameters=[msg_type], return_annotation=msg_type.annotation)

    mirror_func.__signature__ = sig  # type: ignore
    return mirror_func

In [ ]:
for topic, (consumer_f, _, _) in app._consumers_store.items():
    mirror = mirror_consumer(topic, consumer_f)
    assert mirror.__name__ == "to_" + topic
    assert inspect.signature(mirror).return_annotation == TestMsg, inspect.signature(
        mirror
    ).return_annotation
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export


@patch
def create_mirrors(self: Tester) -> None:
    for app in self.apps:
        for topic, (consumer_f, _, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(topic, consumer_f)
            mirror_f = self.produces()(mirror_f)  # type: ignore
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, _) in app._producers_store.items():
            mirror_f = mirror_producer(topic, producer_f)
            mirror_f = self.consumes()(mirror_f)  # type: ignore
            setattr(self, mirror_f.__name__, mirror_f)

In [ ]:
async with Tester(app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

In [ ]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

In [ ]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]).using_local_kafka() as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")